In [1]:
import tellurium as te
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pymc as pm

import gzip 
import cloudpickle

import cobra
import os
os.getcwd()

WARNING (aesara.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


'c:\\Users\\user\\Documents\\research\\BMCA-pipeline\\notebooks\\validation_studies'

In [2]:
os.chdir("../..")
from src import antemll, util
import emll
os.chdir("notebooks/validation_studies")

In [3]:
# 2pg was changed to pg2 because of how antimony parses metabolite
# coefficients

wu_ant = """
    PGM: -> pg2; ; 
    ENO: pg2 -> pep; ; 
    PK: pep -> ; ;

"""
r = te.loada(wu_ant)
with open('../../models/sbml/wu_cobra.xml', 'w') as f:
    f.write(r.getCurrentSBML())

In [4]:
N = r.getFullStoichiometryMatrix()
N

       PGM, ENO, PK
pg2 [[   1,  -1,  0],
pep  [   0,   1, -1]]

In [5]:
giersch_data = pd.read_csv('../../data/giersch.csv').astype(float)
giersch_data

,PGM,ENO,PK,2PG,PEP,BPG,ADP,Flux
0,31.0,5.6,46.0,170.0,50.0,13.2,107.0,157.0
1,25.0,12.0,30.0,60.0,103.0,13.2,61.0,87.0
2,25.0,12.0,30.0,57.0,91.0,13.2,83.0,114.0
3,25.0,12.0,30.0,55.0,83.0,13.2,107.0,128.0
4,25.0,12.0,30.0,54.0,76.0,13.2,130.0,136.0
5,25.0,12.0,30.0,40.0,60.0,8.5,107.0,111.0
6,25.0,12.0,30.0,46.0,70.0,10.8,107.0,118.0
7,25.0,12.0,30.0,55.0,81.0,13.2,107.0,122.0
8,25.0,12.0,30.0,59.0,89.0,15.6,107.0,130.0
9,25.0,12.0,30.0,63.0,95.0,18.0,107.0,133.0


In [6]:
e = giersch_data.loc[:, ['PGM', 'ENO', 'PK']]
y = giersch_data.loc[:, ['BPG', 'ADP']]
v = giersch_data.loc[:, ['Flux']]
x = giersch_data.loc[:, ['2PG', 'PEP']]

ref_ind = 1

v_star = v.iloc[ref_ind].values * np.ones(3)
x_star = x.iloc[ref_ind].values
y_star = y.iloc[ref_ind].values
e_star = e.iloc[ref_ind].values

# Normalize to reference values (and drop trivial measurement)
en = e.values / e_star
yn = np.log(y.values / y_star)
xn = np.log(x.values / x_star)
vn = v.values / v_star[0]

In [7]:
os.getcwd()
te.loads('../../models/sbml/wu_cobra.xml')

<roadrunner.RoadRunner() { this = 0000021FB20DBFA0 }>

In [8]:
import libsbml 
def create_Visser_elasticity_matrix(model_file, cobra_smbl, Ex=True):
    """Create an elasticity matrix for metabolites given the model in model.
    E[j,i] represents the elasticity of reaction j for metabolite i.
    stolen from emll.util.create_elasticity_matrix
    """        
    if Ex: # making the Ex matrix
        cobra_ant = model_file

        r = te.loada(cobra_ant)
        r.conservedMoietyAnalysis = True
        model = cobra.io.read_sbml_model(cobra_smbl)

        n_metabolites = len(model.metabolites)
        n_reactions = len(model.reactions)
        array = np.zeros((n_reactions, n_metabolites), dtype=float)

        m_ind = model.metabolites.index
        r_ind = model.reactions.index

        for reaction in model.reactions:
            for metabolite, stoich in reaction.metabolites.items():

                # Reversible reaction, assign all elements to -stoich
                if reaction.reversibility:
                    array[r_ind(reaction), m_ind(metabolite)] = -np.sign(stoich)

                # Irrevesible in forward direction, only assign if met is reactant
                elif ((not reaction.reversibility) & 
                    (reaction.upper_bound > 0) &
                    (stoich < 0)):
                    array[r_ind(reaction), m_ind(metabolite)] = -np.sign(stoich)

                # Irreversible in reverse direction, only assign if met is product
                elif ((not reaction.reversibility) & 
                    (reaction.lower_bound < 0) &
                    (stoich > 0)):
                    array[r_ind(reaction), m_ind(metabolite)] = -np.sign(stoich)

        array = emll.util.create_elasticity_matrix(model)
        array = pd.DataFrame(array, index=r.getReactionIds(), columns=[i.id for i in model.metabolites])
        array = array.loc[:,r.getFloatingSpeciesIds()]
        
    else: # making the Ey matrix
        r = te.loada(model_file)
        r.conservedMoietyAnalysis = True
        
        doc = libsbml.readSBMLFromString(r.getSBML())
        model = doc.getModel()
        
        rxns = r.getReactionIds()
        bd_sp = r.getBoundarySpeciesIds()
        
        array = np.zeros((len(rxns), len(bd_sp)))
        for n in range(len(rxns)): 
            rxn = model.getReaction(n)
            for reactant in range(rxn.getNumReactants()):                 
                sp = rxn.getReactant(reactant).species
                stoich = rxn.getReactant(reactant).getStoichiometry()
                if sp in bd_sp: 
                    array[n, bd_sp.index(sp)] = -np.sign(stoich)
            for prod in range(rxn.getNumProducts()):
                sp = rxn.getProduct(prod).species
                stoich = rxn.getProduct(prod).getStoichiometry()
                if sp in bd_sp: 
                    array[n, bd_sp.index(sp)] = -np.sign(stoich)
        array = pd.DataFrame(array, index=r.getReactionIds(), \
                                columns=r.getBoundarySpeciesIds())

    return array

In [9]:
Ex = create_Visser_elasticity_matrix(wu_ant, '../../models/sbml/wu_cobra.xml')
Ey = create_Visser_elasticity_matrix(wu_ant, '../../models/sbml/wu_cobra.xml', Ex=False)

No objective coefficients in model. Unclear what should be optimized


In [10]:
Ey

""
PGM
ENO
PK


In [11]:
Ey = np.zeros((3, 2))
Ey[0, 0] = 1  # bpg (+) PGM
Ey[2, 1] = 1  # adp (+) PK
Ey

array([[1., 0.],
       [0., 0.],
       [0., 1.]])

In [12]:
class BMCAobj:
    def __init__(self, N, Ex, Ey, v_star, vn, en, xn, yn):
        self.N = N
        self.Ex = Ex
        self.Ey = Ey
        self.v_star = v_star
        self.vn = vn
        self.en = en
        self.xn = xn
        self.yn = yn

In [13]:
BMCA_obj01 = BMCAobj(
    N = N,
    Ex = Ex,
    Ey = Ey,
    v_star = v_star,
    vn = vn,
    en = en,
    xn = xn,
    yn = yn
)

## Run ADVI

In [ ]:
with pm.Model() as pymc_model:
    
    # Initialize elasticities
    Ex_t = pm.Deterministic('Ex', util.initialize_elasticity(Ex.to_numpy(), name='Ex'))
    Ey_t = pm.Deterministic('Ey', util.initialize_elasticity(Ey, name='Ey'))
    e_obs = pm.Normal('e_obs', mu=1, sigma=1, observed=en.T)
    chi_obs = pm.Normal('chi_obs', mu=0, sigma=10, observed=xn.T)
    y_obs = pm.Normal('y_obs', mu=0, sigma=10, observed=yn.T)
    likelihood = pm.Deterministic('vn', (e_obs * (np.ones(en.T.shape) + pm.math.dot(Ex_t,chi_obs) + pm.math.dot(Ey_t,y_obs)))[2])
    v_hat_obs = pm.Normal('v_hat_obs', mu=likelihood, sigma=0.1, observed=vn.squeeze().T)

    N_ITERATIONS = 45000

with pymc_model:
    advi = pm.ADVI()
    tracker = pm.callbacks.Tracker(
        mean = advi.approx.mean.eval,
        std = advi.approx.std.eval
    )
    approx = advi.fit(
        n= N_ITERATIONS, 
        callbacks = [tracker],
        obj_optimizer=pm.adagrad_window(learning_rate=5E-3), 
        total_grad_norm_constraint=0.7,
        obj_n_mc=1)

with sns.plotting_context('notebook', font_scale=1.2):

    fig = plt.figure(figsize=(5,4))
    plt.plot(approx.hist + 30, '.', rasterized=True, ms=1)
    # plt.ylim([-1E1, 1E3])
    plt.xlim([0, N_ITERATIONS])
    sns.despine(trim=True, offset=10)

    plt.ylabel('-ELBO')
    plt.xlabel('Iteration')
    plt.title('in vitro ADVI convergence')
    plt.tight_layout()

with pymc_model:
    trace = approx.sample(1000)
    ppc_vi = pm.sample_posterior_predictive(trace)


c:\Users\user\miniconda3\envs\gayles\lib\site-packages\aesara\scalar\basic.py:2925: RuntimeWarning:

invalid value encountered in log



In [ ]:
with gzip.open("../../data/results/tracePickles/wu2004_validation.pgz", "wb") as f:
            cloudpickle.dump(
                {
                "trace":trace,
                "ppc":ppc
                },
                f,
            )

## Analysis

In [ ]:
picklefolder = '../../data/results/tracePickles/'
with gzip.open(picklefolder + 'wu2004_validation.pgz', "rb") as f:
    traces = cloudpickle.load(f, encoding='bytes')
trace=traces['trace']

In [ ]:
Ex_advi = util.get_az_summary(trace)
Ey_advi = util.get_az_summary(trace, elasticity='Ey')

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


In [ ]:
np.concatenate((Ex_advi,Ey_advi))

array([-0.769,  0.011,  0.849, -0.771, -0.127,  0.125,  0.842,  0.   ,
       -0.001, -0.002,  0.004,  0.596])

In [27]:
elasticities_df = pd.DataFrame(Ex_advi, columns=['predicted'], 
                               index=[i + '_' + ii for i in r.getReactionIds() for ii in r.getFloatingSpeciesIds()])
elasticities_df 

,predicted
PGM_pg2,-0.769
PGM_pep,0.011
ENO_pg2,0.849
ENO_pep,-0.771
PK_pg2,-0.127
PK_pep,0.125


In [18]:
def run_prior_predictive(BMCA_obj):
    ll = emll.LinLogLeastNorm(N, BMCA_obj.Ex, BMCA_obj.Ey, BMCA_obj.v_star, driver='gelsy')
    
    with pm.Model() as pymc_model:
        # Initialize elasticities
        # Ex and Ey have to be shape (rxns, mets)
        Ex_t = pm.Deterministic('Ex', util.initialize_elasticity(BMCA_obj.Ex.to_numpy().T, 'Ex'))
        Ey_t = pm.Deterministic('Ey', util.initialize_elasticity(BMCA_obj.Ey.T, 'Ey'))

        trace_prior = pm.sample_prior_predictive()

    return trace_prior



In [19]:
Ex01_prior = util.get_az_summary(run_prior_predictive(BMCA_obj01)['prior']['Ex'])

Sampling: [Ex_capacity_entries, Ex_kinetic_entries, Ey_capacity_entries, Ey_kinetic_entries]
arviz - WARNING - Shape validation failed: input_shape: (1, 500), minimum_shape: (chains=2, draws=4)


### Calculating CCCs

In [20]:
priorCCC01_noReg = util.estimate_CCCs(BMCA_obj01, Ex01_prior.reshape((3,-1)))

In [21]:
postCCC01_noReg = util.estimate_CCCs(BMCA_obj01, Ex_advi.reshape((3,-1)))

### Calculating FCCs

In [23]:
priorFCC01 = util.estimate_FCCs(BMCA_obj01, Ex01_prior.reshape((3,2)))

In [24]:
postFCC01 = util.estimate_FCCs(BMCA_obj01, Ex_advi.reshape((3,2)))